In [ ]:
import os
import yt_dlp
import cv2
import torch
import torch.nn as nn
import pandas as pd
from datetime import datetime
from transformers import AutoFeatureExtractor, AutoImageProcessor, AutoModelForImageClassification
from PIL import Image, ImageFile
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

# Función para descargar el último frame del video y clasificar la imagen
def procesar_video_y_clasificar(url, nombre_cámara, carpeta_salida, modelo):
    try:
        # Descargar el último frame del video
        with yt_dlp.YoutubeDL({}) as ydl:
            info_dict = ydl.extract_info(url, download=False)
            video_url = info_dict['formats'][-1]['url']
            cap = cv2.VideoCapture(video_url)
            cap.set(cv2.CAP_PROP_POS_AVI_RATIO, 1)
            _, frame = cap.read()

        # Carpeta de salida para las imágenes
        output_folder = os.path.join(carpeta_salida, str(datetime.now().year), str(datetime.now().month), str(datetime.now().day))

        # Guardar el último frame como una imagen
        if frame is not None:
            os.makedirs(output_folder, exist_ok=True)
            filename = f"{datetime.now().strftime('%H-%M-%S')}_{nombre_cámara}.png"
            output_file = os.path.join(output_folder, filename)
            cv2.imwrite(output_file, frame)
            print(f"Último frame capturado y guardado como {output_file}")

            # Clasificar la imagen
            image_processor = AutoImageProcessor.from_pretrained(modelo)
            model = AutoModelForImageClassification.from_pretrained(modelo)#.to("cuda")
            image = Image.open(output_file)
            input_image = image_processor(images=image, return_tensors="pt")["pixel_values"]#.to("cuda")
            outputs = model(input_image)
            logits = outputs.logits
            probabilities = nn.functional.softmax(logits, dim=-1)
            probabilities = probabilities.detach().cpu().numpy().flatten()
            predicted_class_idx = logits.argmax(-1).item()
            predicted_class = model.config.id2label[predicted_class_idx]
            print("Predicted class:", predicted_class)
            print("Probabilities:", probabilities)

            # Crear el DataFrame
            data = {
                "path_img": [output_file],
                "predicted_class": [predicted_class],
                "probabilities": probabilities.tolist()
            }
            df = pd.DataFrame(data)
            print(df)

            return df
        else:
            print("Error al capturar el último frame")
            return None
    except Exception as e:
        print(f"Error: {e}")
        return None

# Diccionario de nombres de cámaras y URLs
names = {
    'https://www.youtube.com/watch?v=8P2fSKCTen4': 'sannicolas',
    'https://www.youtube.com/watch?v=k8lfQpJRAkY': 'amecameca',
    'https://www.youtube.com/watch?v=4GHVptzyc7g': 'hpanoaya',
    'https://www.youtube.com/watch?v=wuWswhuA_AA': 'tlamacas',
    'https://www.youtube.com/watch?v=517W6mhzPRE': 'tianguismanalco',
    'https://www.youtube.com/watch?v=uMqMPorl7Xs': 'altzomoni'
}

# URLs de las cámaras web
urls_camaras_web = list(names.keys())

# Modelo de clasificación de imágenes
## modelo = "/media/alex/Cold Vault/MimModels/swinv2-base-patch4-window16-256-popo" LINUX
modelo = "A:\\MimModels/swinv2-base-patch4-window16-256-popo" #Windows

# Carpeta de salida para las imágenes
output_folder = "resultados"

# DataFrame para almacenar los resultados
df_final = pd.DataFrame()

# Procesar cada URL de la cámara web
for url in urls_camaras_web:
    nombre_cámara = names[url]
    df = procesar_video_y_clasificar(url, nombre_cámara, output_folder, modelo)
    if df is not None:
        df_final = pd.concat([df_final, df], ignore_index=True)

# Guardar el DataFrame como CSV
df_final.to_csv("resultados.csv", index=False)
print("Resultados guardados como resultados.csv")